In [1]:
import pandas as pd
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.charts import Scatter

from bokeh.layouts import row, column, widgetbox
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, Range1d
from bokeh.models.widgets import Slider, Select

In [2]:
# output_notebook()

In [3]:
path_in = './data/'  # input path
df = pd.read_csv( path_in+'goals_perSeason.csv' )
# team name in all leagues
teams_all = pd.read_csv( path_in+'teams_all.csv' )['teams_all'].tolist()
# team name in a league
teams_league = {'SP1':[],'E0':[],'D1':[],'I1':[],'F1':[]}
for item in teams_league:
    teams_league[item] = pd.read_csv( path_in+'teams_'+item+'.csv' )['teams_league'].tolist()

In [4]:
league_list = ['Spanish La Liga','English Premier League','German BundesLiga','Italian Serie-A','French Ligue-1']

In [5]:
df_plot_ratio = df
df_plot_ratio['y'] = df_plot_ratio['Barcelona-ratio_goal_against']
source = ColumnDataSource(data=df_plot_ratio)

## Plot time-series of goal ratio: scored/against

In [6]:
TOOLS = "pan,wheel_zoom,reset,save"
# color_opt = ['#BD1550','#F8CA00','green','red','blue','black','cyan','magenta']
plot_properties = dict(tools=TOOLS, x_axis_label='time', y_axis_label='scored / against', background_fill_color="#E8DDCB")
# x_axis_type="datetime", 

p = figure(title = 'Goal ratio (per game)', **plot_properties)
p.plot_width = 500
p.plot_height = 500
p.axis.axis_label_standoff = 10
p.axis.axis_label_text_font_size = '20pt'
p.title.text_font_size = '16pt'

In [7]:
# baseline of the plot - all teams' performance
for teamName in teams_all:
    p.line( df['season'].values, df[teamName+'-ratio_goal_against'].values, color='gray', line_width=0.5 )

In [8]:
# Selected team - interactive component
p.line('season', 'y', source=source, color='#BD1550', line_width=3)

# this callback updates the plot
callback_plot = CustomJS(args=dict(source=source), code="""
    var data = source.get('data');
    var f = cb_obj.get('value')
    
    data['y'] = data[f+'-ratio_goal_against'];
    source.trigger('change');  
""")

s1 = Select(title="League:", value="Spanish La Liga", options=league_list)
s2 = Select(title="Teams in the league:", value="Barcelona", options=teams_league['SP1'], callback=callback_plot)

# this callback updates the list in Select-2
s1.callback = CustomJS(args=dict(s1=s1, s2=s2), code="""
      var f = s1.get('value');
      if (f == 'Spanish La Liga') 
        {s2.set('options', ['Alaves','Albacete','Almeria','Ath Bilbao','Ath Madrid','Barcelona','Betis','Cadiz','Celta','Compostela','Cordoba','Eibar','Elche','Espanol','Extremadura','Getafe','Gimnastic','Granada','Hercules','La Coruna','Las Palmas','Lerida','Levante','Logrones','Malaga','Mallorca','Merida','Murcia','Numancia','Osasuna','Oviedo','Real Madrid','Recreativo','Salamanca','Santander','Sevilla','Sociedad','Sp Gijon','Tenerife','Valencia','Valladolid','Vallecano','Villareal','Villarreal','Xerez','Zaragoza'])}
      if (f == 'English Premier League') 
        {s2.set('options', ['Arsenal','Aston Villa','Barnsley','Birmingham','Blackburn','Blackpool','Bolton','Bournemouth','Bradford','Burnley','Cardiff','Charlton','Chelsea','Coventry','Crystal Palace','Derby','Everton','Fulham','Hull','Ipswich','Leeds','Leicester','Liverpool','Man City','Man United','Middlesboro','Middlesbrough','Newcastle','Norwich',"Nott'm Forest",'Oldham','Portsmouth','QPR','Reading','Sheffield United','Sheffield Weds','Southampton','Stoke','Sunderland','Swansea','Swindon','Tottenham','Watford','West Brom','West Ham','Wigan','Wimbledon','Wolves'])}
      if (f == 'German BundesLiga') 
        {s2.set('options', ['Aachen','Augsburg','Bayern Munich','Bielefeld','Bochum','Braunschweig','Cottbus','Darmstadt','Dortmund','Dresden','Duisburg','Dusseldorf','Ein Frankfurt','FC Koln','Fortuna Dusseldorf','Freiburg','Greuther Furth','Hamburg','Hannover','Hansa Rostock','Hertha','Hoffenheim','Ingolstadt','Kaiserslautern','Karlsruhe','Leipzig','Leverkusen',"M'Gladbach","M'gladbach",'Mainz','Munich 1860','Nurnberg','Paderborn','Schalke 04','St Pauli','Stuttgart','Uerdingen','Ulm','Unterhaching','Wattenscheid','Werder Bremen','Wolfsburg'])}
      if (f == 'Italian Serie-A') 
        {s2.set('options', teams_league['Ancona','Ascoli','Atalanta','Bari','Bologna','Brescia','Cagliari','Carpi','Catania','Cesena','Chievo','Como','Cremonese','Empoli','Fiorentina','Foggia','Frosinone','Genoa','Inter','Juventus','Lazio','Lecce','Livorno','Messina','Milan','Modena','Napoli','Novara','Padova','Palermo','Parma','Perugia','Pescara','Piacenza','Reggiana','Reggina','Roma','Salernitana','Sampdoria','Sassuolo','Siena','Torino','Treviso','Udinese','Venezia','Verona','Vicenza'])}
      if (f == 'French Ligue-1') 
        {s2.set('options', teams_league['Ajaccio','Ajaccio GFCO','Angers','Arles','Auxerre','Bastia','Bordeaux','Boulogne','Brest','Caen','Cannes','Chateauroux','Dijon','Evian Thonon Gaillard','Grenoble','Gueugnon','Guingamp','Istres','Le Havre','Le Mans','Lens','Lille','Lorient','Lyon','Marseille','Martigues','Metz','Monaco','Montpellier','Nancy','Nantes','Nice','Paris SG','Reims','Rennes','Sedan','Sochaux','St Etienne','Strasbourg','Toulouse','Troyes','Valenciennes'])}
""")

# show ( row(p, column(s1,s2)) )

## Scatter plot of goal scored v.s. goal against

In [9]:
df_plot_scatter = df
df_plot_scatter['x'] = df_plot_scatter['Barcelona-goalPerGame']
df_plot_scatter['y'] = df_plot_scatter['Barcelona-againstPerGame']
df_plot_scatter['x_point'] = df_plot_scatter['Barcelona-goalPerGame'][22]
df_plot_scatter['y_point'] = df_plot_scatter['Barcelona-againstPerGame'][22]
source_scatter = ColumnDataSource(data=df_plot_scatter)

In [10]:
for leagueName in teams_league:
    df_plot_scatter[leagueName] = 0
    for teamRun in teams_league[leagueName]:
        df_plot_scatter[leagueName] += df_plot_scatter[teamRun+'-goalPerGame']
        df_plot_scatter[leagueName] += df_plot_scatter[teamRun+'-againstPerGame']
    df_plot_scatter[leagueName] = df_plot_scatter[leagueName] / len(teams_league[leagueName])

### Start plotting 

In [11]:
hover = HoverTool(tooltips=[("Year", "@season"),("Scored", "@x"),("Against", "@y")])

In [12]:
TOOLS = [hover] #"pan,wheel_zoom,reset,save,hover" #"pan,wheel_zoom,reset,save,hover"
# color_opt = ['#BD1550','#F8CA00','green','red','blue','black','cyan','magenta']
plot_properties = dict(tools=TOOLS, x_axis_label='goal scored', y_axis_label='goal against', background_fill_color="#E8DDCB")
# x_axis_type="datetime", 

p2 = figure(title = 'Goal scored v.s. against (per game)', **plot_properties)
p2.plot_width = 500
p2.plot_height = 500
p2.axis.axis_label_standoff = 10
p2.axis.axis_label_text_font_size = '20pt'
p2.title.text_font_size = '16pt'

p2.x_range = Range1d(1, 3.5)
p2.y_range = Range1d(0.25, 1.75)

symbolSize = [x**1.8/6 for x in xrange(1,24)]

In [13]:
p2.circle(df_plot_scatter['SP1'], df_plot_scatter['SP1'], color='#1f77b4', size=1, alpha = 0.8)
p2.circle(df_plot_scatter['E0'], df_plot_scatter['E0'], color='#ff7f0e', size=1, alpha = 0.8)
p2.circle(df_plot_scatter['D1'], df_plot_scatter['D1'], color='#2ca02c', size=1, alpha = 0.8)
p2.circle(df_plot_scatter['I1'], df_plot_scatter['I1'], color='#9467bd', size=1, alpha = 0.8)
p2.circle(df_plot_scatter['F1'], df_plot_scatter['F1'], color='gray', size=1, alpha = 0.8)

GlyphRenderer(id='bbf5c8c2-da79-4995-89a8-6fb923c48ea5', ...)

In [14]:
# Selected team - interactive component
p2.circle('x', 'y', source=source_scatter, color='#BD1550', size=symbolSize)
p2.line('x', 'y', source=source_scatter, color='#BD1550', line_width=1)


# this callback updates the plot
callback_plot = CustomJS(args=dict(source=source_scatter), code="""
    var data = source.get('data');
    var f = cb_obj.get('value')
    
    data['x'] = data[f+'-goalPerGame'];
    data['y'] = data[f+'-againstPerGame'];
    source.trigger('change');  
""")

sss1 = Select(title="League:", value="Spanish La Liga", options=league_list)
sss2 = Select(title="Teams in the league:", value="Barcelona", options=teams_league['SP1'], callback=callback_plot)

# this callback updates the list in Select-2
sss1.callback = CustomJS(args=dict(sss1=sss1, sss2=sss2), code="""
      var f = sss1.get('value');
      if (f == 'Spanish La Liga') 
        {sss2.set('options', ['Alaves','Albacete','Almeria','Ath Bilbao','Ath Madrid','Barcelona','Betis','Cadiz','Celta','Compostela','Cordoba','Eibar','Elche','Espanol','Extremadura','Getafe','Gimnastic','Granada','Hercules','La Coruna','Las Palmas','Lerida','Levante','Logrones','Malaga','Mallorca','Merida','Murcia','Numancia','Osasuna','Oviedo','Real Madrid','Recreativo','Salamanca','Santander','Sevilla','Sociedad','Sp Gijon','Tenerife','Valencia','Valladolid','Vallecano','Villareal','Villarreal','Xerez','Zaragoza'])}
      if (f == 'English Premier League') 
        {sss2.set('options', ['Arsenal','Aston Villa','Barnsley','Birmingham','Blackburn','Blackpool','Bolton','Bournemouth','Bradford','Burnley','Cardiff','Charlton','Chelsea','Coventry','Crystal Palace','Derby','Everton','Fulham','Hull','Ipswich','Leeds','Leicester','Liverpool','Man City','Man United','Middlesboro','Middlesbrough','Newcastle','Norwich',"Nott'm Forest",'Oldham','Portsmouth','QPR','Reading','Sheffield United','Sheffield Weds','Southampton','Stoke','Sunderland','Swansea','Swindon','Tottenham','Watford','West Brom','West Ham','Wigan','Wimbledon','Wolves'])}
      if (f == 'German BundesLiga') 
        {sss2.set('options', ['Aachen','Augsburg','Bayern Munich','Bielefeld','Bochum','Braunschweig','Cottbus','Darmstadt','Dortmund','Dresden','Duisburg','Dusseldorf','Ein Frankfurt','FC Koln','Fortuna Dusseldorf','Freiburg','Greuther Furth','Hamburg','Hannover','Hansa Rostock','Hertha','Hoffenheim','Ingolstadt','Kaiserslautern','Karlsruhe','Leipzig','Leverkusen',"M'Gladbach","M'gladbach",'Mainz','Munich 1860','Nurnberg','Paderborn','Schalke 04','St Pauli','Stuttgart','Uerdingen','Ulm','Unterhaching','Wattenscheid','Werder Bremen','Wolfsburg'])}
      if (f == 'Italian Serie-A') 
        {sss2.set('options', teams_league['Ancona','Ascoli','Atalanta','Bari','Bologna','Brescia','Cagliari','Carpi','Catania','Cesena','Chievo','Como','Cremonese','Empoli','Fiorentina','Foggia','Frosinone','Genoa','Inter','Juventus','Lazio','Lecce','Livorno','Messina','Milan','Modena','Napoli','Novara','Padova','Palermo','Parma','Perugia','Pescara','Piacenza','Reggiana','Reggina','Roma','Salernitana','Sampdoria','Sassuolo','Siena','Torino','Treviso','Udinese','Venezia','Verona','Vicenza'])}
      if (f == 'French Ligue-1') 
        {sss2.set('options', teams_league['Ajaccio','Ajaccio GFCO','Angers','Arles','Auxerre','Bastia','Bordeaux','Boulogne','Brest','Caen','Cannes','Chateauroux','Dijon','Evian Thonon Gaillard','Grenoble','Gueugnon','Guingamp','Istres','Le Havre','Le Mans','Lens','Lille','Lorient','Lyon','Marseille','Martigues','Metz','Monaco','Montpellier','Nancy','Nantes','Nice','Paris SG','Reims','Rennes','Sedan','Sochaux','St Etienne','Strasbourg','Toulouse','Troyes','Valenciennes'])}
""")

# show ( row(p2, column(sss1,sss2)) )

/Users/Kunlun/anaconda2/lib/python2.7/site-packages/bokeh/util/deprecation.py:33: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [15]:
show ( column( row(p, column(s1,s2)), row(p2, column(sss1,sss2)) ) )